In [1]:
from moviepy.editor import VideoFileClip
import numpy as np
import os
from datetime import timedelta 
SAVING_FRAMES_PER_SECOND = 20
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

In [2]:
def videoImages(video_file):
    # load the video clip
    video_clip = VideoFileClip(video_file)
    # make a folder by the name of the video file
    filename, _ = os.path.splitext(video_file)
    filename += "-moviepy"
    if not os.path.isdir(filename):
        os.mkdir(filename)

    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(video_clip.fps, SAVING_FRAMES_PER_SECOND)
    # if SAVING_FRAMES_PER_SECOND is set to 0, step is 1/fps, else 1/SAVING_FRAMES_PER_SECOND
    step = 1 / video_clip.fps if saving_frames_per_second == 0 else 1 / saving_frames_per_second
    # iterate over each possible frame
    for current_duration in np.arange(0, video_clip.duration, step):
        # format the file name and save it
        frame_duration_formatted = format_timedelta(timedelta(seconds=current_duration)).replace(":", "-")
        frame_filename = os.path.join(filename, f"frame{frame_duration_formatted}.jpg")
        # save the frame with the current duration
        video_clip.save_frame(frame_filename, current_duration)

In [3]:
videoImages("./andrea_raw.MOV")

In [4]:
import sys

import imageio 
from scipy.linalg import norm
from scipy import sum, average


In [5]:
def compare_images(img1, img2):
    # normalize to compensate for exposure difference, this may be unnecessary
    # consider disabling it
    #img1 = normalize(img1)
    #img2 = normalize(img2)
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    return (m_norm, z_norm)
def to_grayscale(arr):
    "If arr is a color image (3D array), convert it to grayscale (2D array)."
    if len(arr.shape) == 3:
        return average(arr, -1)  # average over the last axis (color channels)
    else:
        return arr
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

In [8]:
def get_clipped_video(video, t_start, t_end):
    '''
    Takes start and end times (in seconds) for video and returns clipped
    video and saves it locally
    Times can be expressed in seconds (15.35), in (min, sec), in (hour, min, sec), or as a string: ‘01:03:05.35’
    '''
    print("made")
    clip = video.subclip(t_start, t_end)
    if clip.rotation == 90:
        clip = clip.resize(clip.size[::-1])
        clip.rotation = 0
    clip.write_videofile("clipped-{}-{}.MOV".format(t_start, t_end),
                              temp_audiofile='temp-audio.m4a', remove_temp=True, codec="libx264", audio_codec="aac")
def make_comparisons(file1, file2):
    # read images as 2D arrays (convert to grayscale for simplicity)
    img1 = to_grayscale(imageio.imread(file1).astype(float))
    img2 = to_grayscale(imageio.imread(file2).astype(float))
    # compare
    n_m, n_0 = compare_images(img1, img2)
    #print(img1.size)
    #print ("Manhattan norm:", n_m, "/ per pixel:", n_m/img1.size)
    #print("Zero norm:", n_0, n_0 * 1.0/img1.size)
    #print(n_0*1.0/img1.size)
    #if n_0*1.0/img1.size < .31:
        #print(n_0)
    return n_0*1.0/img1.size
    #if n_m/img1.size < :
        #print("manhattan less than 1")

In [11]:
from PIL import Image, ImageChops
directory = 'andrea_raw-moviepy'
 
with VideoFileClip("./andrea_raw.MOV") as clip:
    
# that directory
    comparisons = []
    files = os.listdir(directory)
    files.sort()
    curr = -1
    for i in range(1, len(files)):
        filename =files[i]
        f1 = os.path.join(directory, filename)
        filename = files[i - 1]
        f2 = os.path.join(directory, filename)
        #f1 = Image.open(f1)
        #f2 = Image.open(f2)
        comparisons.append(make_comparisons(f1,f2))
    s = 0
    for i in range(len(comparisons)):
        s += comparisons[i]
    avg = s/len(comparisons)
    print("start")
    for i in range(len(comparisons)):
        if comparisons[i] < avg * 1.05:
            if curr == -1:
                curr = i -1
        else:
            if curr != -1 and i - curr >= 20:
                get_clipped_video(clip, .05 *curr,.05* i) # iterate over files in
            curr = -1
    if curr != -1 and len(comparisons) - curr >= 20:
        print("used")
        get_clipped_video(clip, .05 *curr,.05* i)
    print(curr)

KeyboardInterrupt: 